<a href="https://colab.research.google.com/github/ykitaguchi77/statistics_for_articles/blob/main/GravAI_treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**GravAI Before-After study**

In [1]:
# prompt: gdrive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu
import numpy as np

def run_robust_statistical_analysis(file_path):
    """
    より堅牢なデータ処理方法で統計解析を実行する。
    """
    # ===============================================================
    # Step 1: データの読み込みと前処理
    # ===============================================================
    print(f"ファイル '{file_path}' を読み込んでいます...")
    df = pd.read_excel(file_path)

    # フィルター条件は提供されたコードに合わせて一旦コメントアウト
    # df_filtered = df[(df['steroid pulse'] != 0) & (df['operation'] == '-')].copy()
    df_clean = df.copy()

    # --- 安全なデータ抽出関数 ---
    def safe_parse_eye_values(df_to_parse, column_name):
        right_eye_data, left_eye_data = {}, {}
        for idx, value in df_to_parse[column_name].dropna().items():
            if not isinstance(value, str): continue
            parts = value.replace(' ', '').split(',')
            if len(parts) == 2:
                try:
                    right_eye_data[idx] = float(parts[0])
                    left_eye_data[idx] = float(parts[1])
                except (ValueError, TypeError): continue
        s_right = pd.Series(right_eye_data, name=f'{column_name}_R').reindex(df_to_parse.index)
        s_left = pd.Series(left_eye_data, name=f'{column_name}_L').reindex(df_to_parse.index)
        return s_right, s_left

    # --- 全ての列を最初にクリーニング ---
    print("データをクリーニングし、計算用の列を準備しています...")
    # 眼関連の列を抽出し、左右の列を生成
    eye_cols = ['MRD-1_pre', 'MRD1_post', 'MRD-2_pre', 'MRD2_post', 'Hertel_pre', 'Hertel_post']
    for col in eye_cols:
        if col in df_clean.columns:
            s_right, s_left = safe_parse_eye_values(df_clean, col)
            df_clean[s_right.name] = s_right
            df_clean[s_left.name] = s_left

    # 非眼関連の列を安全に数値化
    non_eye_cols = ['Gorman score_pre', 'Gorman score_post', 'CAS_pre', 'CAS_post']
    for col in non_eye_cols:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    # --- 正しい方法でΔ（変化量）を計算 ---
    # 患者ごと（行ごと）に計算し、ペアがなければNaNになる
    delta_definitions = {
        'ΔMRD-1': ('MRD-1_pre', 'MRD1_post'),
        'ΔMRD-2': ('MRD-2_pre', 'MRD2_post'),
        'ΔHertel': ('Hertel_pre', 'Hertel_post'),
    }
    for delta_name, (pre_name, post_name) in delta_definitions.items():
        # post_nameのハイフンを削除 (例: MRD1_post)
        post_name_clean = post_name.replace('-', '')
        if f'{pre_name}_R' in df_clean.columns and f'{post_name_clean}_R' in df_clean.columns:
            df_clean[f'{delta_name}_R'] = df_clean[f'{post_name_clean}_R'] - df_clean[f'{pre_name}_R']
            df_clean[f'{delta_name}_L'] = df_clean[f'{post_name_clean}_L'] - df_clean[f'{pre_name}_L']

    # 非眼関連のΔを計算
    if 'Gorman score_pre' in df_clean.columns and 'Gorman score_post' in df_clean.columns:
        df_clean['ΔGorman score'] = df_clean['Gorman score_post'] - df_clean['Gorman score_pre']
    if 'CAS_pre' in df_clean.columns and 'CAS_post' in df_clean.columns:
        df_clean['ΔCAS'] = df_clean['CAS_post'] - df_clean['CAS_pre']

    # ===============================================================
    # Step 2: 統計解析の実行
    # ===============================================================
    print("統計解析を実行しています...")

    # --- ヘルパー関数 (再利用) ---
    def descriptive_stats(series):
        series_clean = series.dropna()
        if len(series_clean) == 0:
            return {'Min': None, 'Q1': None, 'Median': None, 'Q3': None, 'Max': None, 'Mean': None, 'SD': None, 'Count': 0}
        return {'Min': series_clean.min(), 'Q1': series_clean.quantile(0.25), 'Median': series_clean.median(),
                'Q3': series_clean.quantile(0.75), 'Max': series_clean.max(), 'Mean': series_clean.mean(),
                'SD': series_clean.std(), 'Count': len(series_clean)}

    def mann_whitney_test(group1, group2):
        g1_clean, g2_clean = group1.dropna(), group2.dropna()
        if len(g1_clean) > 0 and len(g2_clean) > 0:
            return mannwhitneyu(g1_clean, g2_clean, alternative='two-sided')
        return None, None

    # --- 解析の準備 ---
    results = []
    # 解析対象の変数を定義
    variables_to_analyze = [
        'MRD-1_pre_R', 'MRD-1_pre_L', 'MRD1_post_R', 'MRD1_post_L', 'ΔMRD-1_R', 'ΔMRD-1_L',
        'MRD-2_pre_R', 'MRD-2_pre_L', 'MRD2_post_R', 'MRD2_post_L', 'ΔMRD-2_R', 'ΔMRD-2_L',
        'Hertel_pre_R', 'Hertel_pre_L', 'Hertel_post_R', 'Hertel_post_L', 'ΔHertel_R', 'ΔHertel_L',
        'Gorman score_pre', 'Gorman score_post', 'ΔGorman score',
        'CAS_pre', 'CAS_post', 'ΔCAS'
    ]

    # 2つのグループを定義
    group1_df = df_clean[df_clean['difference'] <= -30]
    group2_df = df_clean[df_clean['difference'] > -30]
    print(f"\ndifference ≤ -30群: {len(group1_df)}件")
    print(f"difference > -30群: {len(group2_df)}件")

    # --- ループによる解析実行 ---
    for var in variables_to_analyze:
        if var not in df_clean.columns: continue

        # データを抽出
        g1_series = group1_df[var]
        g2_series = group2_df[var]

        # Mann-Whitney U検定
        stat, p_value = mann_whitney_test(g1_series, g2_series)

        # 結果の整形
        # Variable名から _R, _L を取り除く
        var_name = var.replace('_R', '').replace('_L', '')
        eye_type = 'Right' if var.endswith('_R') else 'Left' if var.endswith('_L') else 'Both'

        # 結果をリストに追加
        for group_name, series in [('difference ≤ -30', g1_series), ('difference > -30', g2_series)]:
            stats = descriptive_stats(series)
            results.append({'Variable': var_name, 'Eye': eye_type, 'Group': group_name, 'Statistic': stat, 'P-value': p_value, **stats})

    # Bilateral (両眼) のデータを追加
    bilateral_vars = [v for v in variables_to_analyze if v.endswith('_R')]
    for var_r in bilateral_vars:
        var_l = var_r.replace('_R', '_L')
        if var_l in df_clean.columns:
            # 両眼のデータを作成
            g1_bilateral = pd.concat([group1_df[var_r], group1_df[var_l]], ignore_index=True)
            g2_bilateral = pd.concat([group2_df[var_r], group2_df[var_l]], ignore_index=True)

            stat, p_value = mann_whitney_test(g1_bilateral, g2_bilateral)
            var_name = var_r.replace('_R', '')

            for group_name, series in [('difference ≤ -30', g1_bilateral), ('difference > -30', g2_bilateral)]:
                stats = descriptive_stats(series)
                results.append({'Variable': var_name, 'Eye': 'Bilateral', 'Group': group_name, 'Statistic': stat, 'P-value': p_value, **stats})

    # ===============================================================
    # Step 3: 結果の出力
    # ===============================================================
    print("\n===== 統計分析結果 =====")
    results_df = pd.DataFrame(results)

    # 表示形式を整える
    pd.options.display.float_format = '{:.4f}'.format

    # Significance列を追加
    results_df['Significance'] = np.where(results_df['P-value'] < 0.05, 'Significant', 'Not Significant')
    print(results_df)

    # Excelに保存
    output_filename = 'statistical_analysis_results_robust.xlsx'
    results_df.to_excel(output_filename, index=False)
    print(f"\nExcelファイル '{output_filename}' に結果を保存しました。")

    # ===== Mean±SD形式の結果を出力 =====
    print("\n===== Mean±SD形式の結果 =====")
    # pivot_tableでデータを整形
    summary_table = results_df.pivot_table(
        index=['Variable', 'Eye'],
        columns='Group',
        values=['Mean', 'SD', 'Count', 'P-value']
    ).reset_index()

    # 表示
    for index, row in summary_table.iterrows():
        var, eye = row[('Variable', '')], row[('Eye', '')]
        mean1, sd1, n1 = row[('Mean', 'difference ≤ -30')], row[('SD', 'difference ≤ -30')], row[('Count', 'difference ≤ -30')]
        mean2, sd2, n2 = row[('Mean', 'difference > -30')], row[('SD', 'difference > -30')], row[('Count', 'difference > -30')]
        p_val = row[('P-value', 'difference ≤ -30')] # p-valueはどちらのグループも同じ

        # データが存在する場合のみ表示
        if pd.notna(mean1) and pd.notna(mean2):
            print(f"\n----- {var} ({eye}) -----")
            print(f"  ≤ -30群: {mean1:.2f} ± {sd1:.2f} (n={int(n1)})")
            print(f"  > -30群: {mean2:.2f} ± {sd2:.2f} (n={int(n2)})")
            print(f"  p値: {p_val:.4f} {'(有意差あり)' if p_val < 0.05 else ''}")

    # ===== 有意差のある項目を最後にまとめる =====
    significant_results = results_df[
        (results_df['P-value'] < 0.05) &
        (results_df['Group'] == 'difference ≤ -30') # 重複を避けるため片方の群のみ選択
    ].sort_values('P-value')

    if not significant_results.empty:
        print("\n===== 有意差のある項目 (p<0.05) =====")
        print(significant_results[['Variable', 'Eye', 'P-value']])
    else:
        print("\n有意差のある項目はありませんでした。")

    return results_df

# ===============================================================
# 実行ブロック
# ===============================================================
# ファイルパスを正確に設定してください
file_path = "/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'_20250612.xlsx"

# 関数を実行
final_results = run_robust_statistical_analysis(file_path)

In [ ]:
### ステロイドパルスのみ群のみをフィルタリング

import pandas as pd
from scipy.stats import mannwhitneyu
import numpy as np

def run_robust_statistical_analysis(file_path):
    """
    より堅牢なデータ処理方法で統計解析を実行する。
    """
    # ===============================================================
    # Step 1: データの読み込みと前処理
    # ===============================================================
    print(f"ファイル '{file_path}' を読み込んでいます...")
    df = pd.read_excel(file_path)

    # フィルター条件は提供されたコードに合わせて一旦コメントアウト
    df_clean = df[(df['steroid pulse'] != 0) & (df['operation'] == '-')].copy()
    #df_clean = df.copy()

    # --- 安全なデータ抽出関数 ---
    def safe_parse_eye_values(df_to_parse, column_name):
        right_eye_data, left_eye_data = {}, {}
        for idx, value in df_to_parse[column_name].dropna().items():
            if not isinstance(value, str): continue
            parts = value.replace(' ', '').split(',')
            if len(parts) == 2:
                try:
                    right_eye_data[idx] = float(parts[0])
                    left_eye_data[idx] = float(parts[1])
                except (ValueError, TypeError): continue
        s_right = pd.Series(right_eye_data, name=f'{column_name}_R').reindex(df_to_parse.index)
        s_left = pd.Series(left_eye_data, name=f'{column_name}_L').reindex(df_to_parse.index)
        return s_right, s_left

    # --- 全ての列を最初にクリーニング ---
    print("データをクリーニングし、計算用の列を準備しています...")
    # 眼関連の列を抽出し、左右の列を生成
    eye_cols = ['MRD-1_pre', 'MRD1_post', 'MRD-2_pre', 'MRD2_post', 'Hertel_pre', 'Hertel_post']
    for col in eye_cols:
        if col in df_clean.columns:
            s_right, s_left = safe_parse_eye_values(df_clean, col)
            df_clean[s_right.name] = s_right
            df_clean[s_left.name] = s_left

    # 非眼関連の列を安全に数値化
    non_eye_cols = ['Gorman score_pre', 'Gorman score_post', 'CAS_pre', 'CAS_post']
    for col in non_eye_cols:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    # --- 正しい方法でΔ（変化量）を計算 ---
    # 患者ごと（行ごと）に計算し、ペアがなければNaNになる
    delta_definitions = {
        'ΔMRD-1': ('MRD-1_pre', 'MRD1_post'),
        'ΔMRD-2': ('MRD-2_pre', 'MRD2_post'),
        'ΔHertel': ('Hertel_pre', 'Hertel_post'),
    }
    for delta_name, (pre_name, post_name) in delta_definitions.items():
        # post_nameのハイフンを削除 (例: MRD1_post)
        post_name_clean = post_name.replace('-', '')
        if f'{pre_name}_R' in df_clean.columns and f'{post_name_clean}_R' in df_clean.columns:
            df_clean[f'{delta_name}_R'] = df_clean[f'{post_name_clean}_R'] - df_clean[f'{pre_name}_R']
            df_clean[f'{delta_name}_L'] = df_clean[f'{post_name_clean}_L'] - df_clean[f'{pre_name}_L']

    # 非眼関連のΔを計算
    if 'Gorman score_pre' in df_clean.columns and 'Gorman score_post' in df_clean.columns:
        df_clean['ΔGorman score'] = df_clean['Gorman score_post'] - df_clean['Gorman score_pre']
    if 'CAS_pre' in df_clean.columns and 'CAS_post' in df_clean.columns:
        df_clean['ΔCAS'] = df_clean['CAS_post'] - df_clean['CAS_pre']

    # ===============================================================
    # Step 2: 統計解析の実行
    # ===============================================================
    print("統計解析を実行しています...")

    # --- ヘルパー関数 (再利用) ---
    def descriptive_stats(series):
        series_clean = series.dropna()
        if len(series_clean) == 0:
            return {'Min': None, 'Q1': None, 'Median': None, 'Q3': None, 'Max': None, 'Mean': None, 'SD': None, 'Count': 0}
        return {'Min': series_clean.min(), 'Q1': series_clean.quantile(0.25), 'Median': series_clean.median(),
                'Q3': series_clean.quantile(0.75), 'Max': series_clean.max(), 'Mean': series_clean.mean(),
                'SD': series_clean.std(), 'Count': len(series_clean)}

    def mann_whitney_test(group1, group2):
        g1_clean, g2_clean = group1.dropna(), group2.dropna()
        if len(g1_clean) > 0 and len(g2_clean) > 0:
            return mannwhitneyu(g1_clean, g2_clean, alternative='two-sided')
        return None, None

    # --- 解析の準備 ---
    results = []
    # 解析対象の変数を定義
    variables_to_analyze = [
        'MRD-1_pre_R', 'MRD-1_pre_L', 'MRD1_post_R', 'MRD1_post_L', 'ΔMRD-1_R', 'ΔMRD-1_L',
        'MRD-2_pre_R', 'MRD-2_pre_L', 'MRD2_post_R', 'MRD2_post_L', 'ΔMRD-2_R', 'ΔMRD-2_L',
        'Hertel_pre_R', 'Hertel_pre_L', 'Hertel_post_R', 'Hertel_post_L', 'ΔHertel_R', 'ΔHertel_L',
        'Gorman score_pre', 'Gorman score_post', 'ΔGorman score',
        'CAS_pre', 'CAS_post', 'ΔCAS'
    ]

    # 2つのグループを定義
    group1_df = df_clean[df_clean['difference'] <= -30]
    group2_df = df_clean[df_clean['difference'] > -30]
    print(f"\ndifference ≤ -30群: {len(group1_df)}件")
    print(f"difference > -30群: {len(group2_df)}件")

    # --- ループによる解析実行 ---
    for var in variables_to_analyze:
        if var not in df_clean.columns: continue

        # データを抽出
        g1_series = group1_df[var]
        g2_series = group2_df[var]

        # Mann-Whitney U検定
        stat, p_value = mann_whitney_test(g1_series, g2_series)

        # 結果の整形
        # Variable名から _R, _L を取り除く
        var_name = var.replace('_R', '').replace('_L', '')
        eye_type = 'Right' if var.endswith('_R') else 'Left' if var.endswith('_L') else 'Both'

        # 結果をリストに追加
        for group_name, series in [('difference ≤ -30', g1_series), ('difference > -30', g2_series)]:
            stats = descriptive_stats(series)
            results.append({'Variable': var_name, 'Eye': eye_type, 'Group': group_name, 'Statistic': stat, 'P-value': p_value, **stats})

    # Bilateral (両眼) のデータを追加
    bilateral_vars = [v for v in variables_to_analyze if v.endswith('_R')]
    for var_r in bilateral_vars:
        var_l = var_r.replace('_R', '_L')
        if var_l in df_clean.columns:
            # 両眼のデータを作成
            g1_bilateral = pd.concat([group1_df[var_r], group1_df[var_l]], ignore_index=True)
            g2_bilateral = pd.concat([group2_df[var_r], group2_df[var_l]], ignore_index=True)

            stat, p_value = mann_whitney_test(g1_bilateral, g2_bilateral)
            var_name = var_r.replace('_R', '')

            for group_name, series in [('difference ≤ -30', g1_bilateral), ('difference > -30', g2_bilateral)]:
                stats = descriptive_stats(series)
                results.append({'Variable': var_name, 'Eye': 'Bilateral', 'Group': group_name, 'Statistic': stat, 'P-value': p_value, **stats})

    # ===============================================================
    # Step 3: 結果の出力
    # ===============================================================
    print("\n===== 統計分析結果 =====")
    results_df = pd.DataFrame(results)

    # 表示形式を整える
    pd.options.display.float_format = '{:.4f}'.format

    # Significance列を追加
    results_df['Significance'] = np.where(results_df['P-value'] < 0.05, 'Significant', 'Not Significant')
    print(results_df)

    # Excelに保存
    output_filename = 'statistical_analysis_results_robust.xlsx'
    results_df.to_excel(output_filename, index=False)
    print(f"\nExcelファイル '{output_filename}' に結果を保存しました。")

    # ===== Mean±SD形式の結果を出力 =====
    print("\n===== Mean±SD形式の結果 =====")
    # pivot_tableでデータを整形
    summary_table = results_df.pivot_table(
        index=['Variable', 'Eye'],
        columns='Group',
        values=['Mean', 'SD', 'Count', 'P-value']
    ).reset_index()

    # 表示
    for index, row in summary_table.iterrows():
        var, eye = row[('Variable', '')], row[('Eye', '')]
        mean1, sd1, n1 = row[('Mean', 'difference ≤ -30')], row[('SD', 'difference ≤ -30')], row[('Count', 'difference ≤ -30')]
        mean2, sd2, n2 = row[('Mean', 'difference > -30')], row[('SD', 'difference > -30')], row[('Count', 'difference > -30')]
        p_val = row[('P-value', 'difference ≤ -30')] # p-valueはどちらのグループも同じ

        # データが存在する場合のみ表示
        if pd.notna(mean1) and pd.notna(mean2):
            print(f"\n----- {var} ({eye}) -----")
            print(f"  ≤ -30群: {mean1:.2f} ± {sd1:.2f} (n={int(n1)})")
            print(f"  > -30群: {mean2:.2f} ± {sd2:.2f} (n={int(n2)})")
            print(f"  p値: {p_val:.4f} {'(有意差あり)' if p_val < 0.05 else ''}")

    # ===== 有意差のある項目を最後にまとめる =====
    significant_results = results_df[
        (results_df['P-value'] < 0.05) &
        (results_df['Group'] == 'difference ≤ -30') # 重複を避けるため片方の群のみ選択
    ].sort_values('P-value')

    if not significant_results.empty:
        print("\n===== 有意差のある項目 (p<0.05) =====")
        print(significant_results[['Variable', 'Eye', 'P-value']])
    else:
        print("\n有意差のある項目はありませんでした。")

    return results_df

# ===============================================================
# 実行ブロック
# ===============================================================
# ファイルパスを正確に設定してください
file_path = "/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'_20250612.xlsx"

# 関数を実行
final_results = run_robust_statistical_analysis(file_path)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib as mpl
import os

# Google Colabでの日本語フォント対応（最初に実行する必要があります）
# 実行時に以下のコマンドをセルで実行してフォントをインストール
# !apt-get -y install fonts-ipafont-gothic fonts-ipafont-mincho

# フォント設定（Google Colab環境用に最適化）
def setup_japanese_fonts():
    # Colab環境でのフォントインストール確認
    if not os.path.exists('/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'):
        print("警告: 日本語フォントが見つかりません。")
        print("以下のコマンドを別セルで実行してください:")
        print("!apt-get -y install fonts-ipafont-gothic fonts-ipafont-mincho")
        print("!pip install japanize-matplotlib")
        # 英語ラベルを使用
        return False

    # フォント設定
    plt.rcParams['font.family'] = 'IPAexGothic, IPAGothic, sans-serif'
    plt.rcParams['font.sans-serif'] = ['IPAexGothic', 'IPAGothic', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['font.size'] = 14

    # Seabornの設定
    sns.set(style="whitegrid", font_scale=1.2)

    # 日本語フォントが使用可能
    return True

# ファイルパスを設定
file_path = "/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'.xlsx"

def create_quartile_plots(file_path):
    # 日本語フォント設定
    use_japanese = setup_japanese_fonts()

    # Excelファイルを読み込む
    print(f"ファイル '{file_path}' を読み込んでいます...")
    df = pd.read_excel(file_path)

    # 元の分析と同様のフィルタリング
    print("データをフィルタリングしています...")
    df_filtered = df[(df['steroid pulse'] != 0) & (df['operation'] == '-')]

    # グループ分け
    group_negative_30_or_less = df_filtered[df_filtered['difference'] <= -30]
    group_other = df_filtered[df_filtered['difference'] > -30]

    print(f"difference ≤ -30群: {len(group_negative_30_or_less)}件")
    print(f"difference > -30群: {len(group_other)}件")

    # 右目と左目の値を分離する関数
    def split_and_convert_eye_values(group, column):
        right_eye, left_eye = [], []
        for value in group[column].dropna():
            if isinstance(value, str):
                parts = value.replace(' ', '').split(',')
                if len(parts) == 2:
                    try:
                        right_eye.append(float(parts[0]))
                        left_eye.append(float(parts[1]))
                    except ValueError:
                        # 数値変換に失敗した場合はスキップ
                        pass
        return pd.Series(right_eye), pd.Series(left_eye)

    # 眼関連の変数と非眼関連の変数を定義（元のコードから取得）
    eye_related_columns = [
        'MRD-1_pre', 'MRD1_post', 'MRD-2_pre', 'MRD2_post',
        'Hertel_pre', 'Hertel_post'
    ]
    non_eye_columns = ['Gorman score_pre', 'Gorman score_post']

    # 変数をグループ化して共通のスケールを設定するための辞書
    variable_groups = {
        'MRD1': ['MRD-1_pre', 'MRD1_post'],
        'MRD2': ['MRD-2_pre', 'MRD2_post'],
        'Hertel': ['Hertel_pre', 'Hertel_post'],
        'Gorman': ['Gorman score_pre', 'Gorman score_post']
    }

    # 日本語表示用の設定
    right_eye_label = "右眼 (R)" if use_japanese else "Right Eye (R)"
    left_eye_label = "左眼 (L)" if use_japanese else "Left Eye (L)"
    value_label = "値" if use_japanese else "Value"

    # 眼関連変数のための図を作成
    print("眼関連変数の4分位図を作成しています...")
    fig_eye, axes_eye = plt.subplots(len(eye_related_columns), 1,
                                     figsize=(12, 5*len(eye_related_columns)))

    # フォント設定の再適用（確実に適用するため）
    for prop in mpl.rcParams:
        if prop.startswith('font'):
            if prop in plt.rcParams:
                plt.rcParams[prop] = mpl.rcParams[prop]

    # 眼関連変数が1つの場合、axesは配列にならないため調整
    if len(eye_related_columns) == 1:
        axes_eye = [axes_eye]

    # 各メトリックタイプのスケール範囲を格納する辞書
    scale_ranges = {}

    # 各メトリックタイプごとにすべてのデータを収集し、スケール範囲を決定
    for group_name, columns in variable_groups.items():
        if group_name == 'Gorman':
            # Gormanスコアは1-4の範囲で固定
            scale_ranges[group_name] = (0.5, 4.5)
            continue

        all_values = []
        eye_columns = [col for col in columns if col in eye_related_columns]

        for column in eye_columns:
            # 全グループ、両眼のデータを収集
            for group in [group_negative_30_or_less, group_other]:
                right_eye, left_eye = split_and_convert_eye_values(group, column)
                all_values.extend(right_eye.values)
                all_values.extend(left_eye.values)

        if all_values:
            # 10%のマージンでスケール範囲を設定
            data_range = max(all_values) - min(all_values)
            margin = data_range * 0.1
            scale_ranges[group_name] = (min(all_values) - margin, max(all_values) + margin)

    # 各眼関連変数をプロット
    for i, column in enumerate(eye_related_columns):
        ax = axes_eye[i]

        # 両グループ、両眼のデータを取得
        right_eye_neg_30, left_eye_neg_30 = split_and_convert_eye_values(group_negative_30_or_less, column)
        right_eye_other, left_eye_other = split_and_convert_eye_values(group_other, column)

        # インデックスの問題を解決するため、新しいシリーズを作成
        right_eye_neg_30 = pd.Series(right_eye_neg_30.values)
        right_eye_other = pd.Series(right_eye_other.values)
        left_eye_neg_30 = pd.Series(left_eye_neg_30.values)
        left_eye_other = pd.Series(left_eye_other.values)

        # プロット用のDataFrameを作成（インデックスをリセット）
        plot_data = pd.DataFrame({
            'Value': pd.concat([right_eye_neg_30, right_eye_other, left_eye_neg_30, left_eye_other], ignore_index=True),
            'Group': pd.concat([
                pd.Series(['≤ -30'] * len(right_eye_neg_30)),
                pd.Series(['> -30'] * len(right_eye_other)),
                pd.Series(['≤ -30'] * len(left_eye_neg_30)),
                pd.Series(['> -30'] * len(left_eye_other))
            ], ignore_index=True),
            'Eye': pd.concat([
                pd.Series([right_eye_label] * len(right_eye_neg_30)),
                pd.Series([right_eye_label] * len(right_eye_other)),
                pd.Series([left_eye_label] * len(left_eye_neg_30)),
                pd.Series([left_eye_label] * len(left_eye_other))
            ], ignore_index=True)
        })

        # 箱ひげ図を作成（seaborn警告に対応）
        sns.boxplot(x='Eye', y='Value', hue='Group', data=plot_data,
                    palette=['#3498db', '#e74c3c'], ax=ax)

        # 個別データポイントを追加（seaborn警告に対応）
        sns.stripplot(x='Eye', y='Value', hue='Group', data=plot_data,
                     dodge=True, alpha=0.5, jitter=True, ax=ax,
                     palette=['#3498db', '#e74c3c'])

        # タイトルとラベルを設定
        ax.set_title(f'{column}', fontsize=14, fontweight='bold')
        ax.set_ylabel(value_label, fontsize=12)
        ax.set_xlabel('')

        # このメトリックのスケール範囲を設定
        for group_name, cols in variable_groups.items():
            if column in cols and group_name in scale_ranges:
                ax.set_ylim(scale_ranges[group_name])
                break

        # 凡例を設定（表示が重複するので最初の2つだけ表示）
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles[:2], labels[:2], title='difference', loc='best')

        # サンプルサイズの注釈を追加
        ax.annotate(f'n(≤-30, R)={len(right_eye_neg_30)}, n(>-30, R)={len(right_eye_other)}\n'
                    f'n(≤-30, L)={len(left_eye_neg_30)}, n(>-30, L)={len(left_eye_other)}',
                    xy=(0.02, 0.02), xycoords='axes fraction', fontsize=10,
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

    plt.tight_layout()

    # 非眼関連変数のための図を作成
    print("非眼関連変数の4分位図を作成しています...")
    fig_non_eye, axes_non_eye = plt.subplots(len(non_eye_columns), 1,
                                           figsize=(10, 5*len(non_eye_columns)))

    # 非眼関連変数が1つの場合、axesは配列にならないため調整
    if len(non_eye_columns) == 1:
        axes_non_eye = [axes_non_eye]

    # 各非眼関連変数をプロット
    for i, column in enumerate(non_eye_columns):
        ax = axes_non_eye[i]

        # 両グループのデータを取得
        data_neg_30 = pd.to_numeric(group_negative_30_or_less[column], errors='coerce').dropna()
        data_other = pd.to_numeric(group_other[column], errors='coerce').dropna()

        # インデックスの問題を解決するため、新しいシリーズを作成
        data_neg_30 = pd.Series(data_neg_30.values)
        data_other = pd.Series(data_other.values)

        # プロット用のDataFrameを作成（インデックスをリセット）
        plot_data = pd.DataFrame({
            'Value': pd.concat([data_neg_30, data_other], ignore_index=True),
            'Group': pd.concat([
                pd.Series(['≤ -30'] * len(data_neg_30)),
                pd.Series(['> -30'] * len(data_other))
            ], ignore_index=True)
        })

        # 箱ひげ図を作成（seaborn警告に対応）
        sns.boxplot(x='Group', y='Value', hue='Group', data=plot_data,
                   palette=['#3498db', '#e74c3c'], ax=ax, legend=False)

        # 個別データポイントを追加（seaborn警告に対応）
        sns.stripplot(x='Group', y='Value', hue='Group', data=plot_data,
                     dodge=True, alpha=0.7, jitter=True, ax=ax,
                     palette=['#3498db', '#e74c3c'], legend=False)

        # タイトルとラベルを設定
        ax.set_title(f'{column}', fontsize=14, fontweight='bold')
        ax.set_ylabel(value_label, fontsize=12)
        ax.set_xlabel('difference', fontsize=12)

        # Gormanスコアのスケール設定（1-4）
        for group_name, cols in variable_groups.items():
            if column in cols and group_name in scale_ranges:
                ax.set_ylim(scale_ranges[group_name])

                # Gormanスコアの場合は整数の目盛りのみを表示
                if group_name == 'Gorman':
                    # 整数の目盛りのみを表示
                    import matplotlib.ticker as ticker
                    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
                    ax.set_yticks([1, 2, 3, 4])
                    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
                break

        # サンプルサイズの注釈を追加
        ax.annotate(f'n(≤-30)={len(data_neg_30)}, n(>-30)={len(data_other)}',
                    xy=(0.02, 0.02), xycoords='axes fraction', fontsize=10,
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

    plt.tight_layout()

    # 画像のファイル名（英語で保存してエラー回避）
    eye_fig_filename = 'eye_variables_boxplot.png'
    non_eye_fig_filename = 'non_eye_variables_boxplot.png'

    # 図を保存
    print("図を保存しています...")
    fig_eye.savefig(eye_fig_filename, dpi=300, bbox_inches='tight')
    fig_non_eye.savefig(non_eye_fig_filename, dpi=300, bbox_inches='tight')

    print("完了しました！以下の2つの図が保存されました：")
    print(f"1. {eye_fig_filename}")
    print(f"2. {non_eye_fig_filename}")

    # 図を表示
    plt.show()

# ファイルパスを指定して関数を実行
if __name__ == "__main__":
    create_quartile_plots(file_path)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy.stats import mannwhitneyu

# ===============================================================
# 1. 日本語フォント設定
# ===============================================================
# !apt-get -y install fonts-ipafont-gothic > /dev/null
# !pip install japanize-matplotlib > /dev/null
import japanize_matplotlib
plt.rcParams['font.sans-serif'] = ['IPAexGothic']
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style="ticks", font='IPAexGothic')

# ===============================================================
# 2. データ前処理とグラフ描画用の関数 (最新の正確な方式)
# ===============================================================

def safe_parse_eye_values(df, column_name):
    """安全に左右の眼の値を抽出し、元のDataFrameのindexに揃えて返す"""
    right_eye_data, left_eye_data = {}, {}
    for idx, value in df[column_name].dropna().items():
        if not isinstance(value, str): continue
        parts = value.replace(' ', '').split(',')
        if len(parts) == 2:
            try:
                right_eye_data[idx] = float(parts[0])
                left_eye_data[idx] = float(parts[1])
            except (ValueError, TypeError): continue
    s_right = pd.Series(right_eye_data, name=f'{column_name}_R').reindex(df.index)
    s_left = pd.Series(left_eye_data, name=f'{column_name}_L').reindex(df.index)
    return s_right, s_left

def preprocess_and_create_variables(file_path):
    """データを前処理し、解析と描画に必要なすべての変数列を作成する"""
    print("データ前処理を開始します...")
    df = pd.read_excel(file_path)
    df_clean = df.copy()
    df_clean['Group'] = np.where(df_clean['difference'] <= -30, '改善群', '対照群')

    eye_cols = ['MRD-1_pre', 'MRD1_post', 'MRD-2_pre', 'MRD2_post', 'Hertel_pre', 'Hertel_post']
    for col in eye_cols:
        if col in df_clean.columns:
            s_right, s_left = safe_parse_eye_values(df_clean, col)
            df_clean[s_right.name] = s_right
            df_clean[s_left.name] = s_left

    delta_definitions = {'ΔMRD-1': ('MRD-1_pre', 'MRD1_post'), 'ΔMRD-2': ('MRD-2_pre', 'MRD2_post'), 'ΔHertel': ('Hertel_pre', 'Hertel_post')}
    for name, (pre, post) in delta_definitions.items():
        post_clean = post.replace('-', '')
        for eye in ['_R', '_L']:
            if f'{pre}{eye}' in df_clean and f'{post_clean}{eye}' in df_clean:
                df_clean[f'{name}{eye}'] = df_clean[f'{post_clean}{eye}'] - df_clean[f'{pre}{eye}']

    for col_name in list(df_clean.columns):
        if col_name.endswith('_R'):
            base_name = col_name[:-2]
            if f'{base_name}_L' in df_clean:
                df_clean[f'{base_name}_Bilateral'] = df_clean.apply(
                    lambda row: list(pd.concat([pd.Series(row[f'{base_name}_R']), pd.Series(row[f'{base_name}_L'])], ignore_index=True).dropna()),
                    axis=1
                )

    for metric in ['Gorman score', 'CAS']:
        if f'{metric}_pre' in df_clean and f'{metric}_post' in df_clean:
            df_clean[f'{metric}_pre'] = pd.to_numeric(df_clean[f'{metric}_pre'], errors='coerce')
            df_clean[f'{metric}_post'] = pd.to_numeric(df_clean[f'{metric}_post'], errors='coerce')
            df_clean[f'Δ{metric}'] = df_clean[f'{metric}_post'] - df_clean[f'{metric}_pre']

    print("データ前処理が完了しました。")
    return df_clean

def create_comparison_boxplot(ax, df, column_name, title):
    """
    2群比較のボックスプロットを作成し、p値と有意性を明確に表示する関数
    """
    if column_name.endswith('_Bilateral'):
        plot_data = df[['Group', column_name]].dropna().explode(column_name)
        plot_data = plot_data.rename(columns={column_name: 'Value'})
        # ★★★ ここが修正点 ★★★
        # 展開(explode)によって重複したインデックスをリセットする
        plot_data = plot_data.reset_index(drop=True)
        plot_data['Value'] = pd.to_numeric(plot_data['Value'])
    else:
        plot_data = df[['Group', column_name]].rename(columns={column_name: 'Value'}).dropna()
        # こちらも念のためインデックスをリセット
        plot_data = plot_data.reset_index(drop=True)


    order = ['改善群', '対照群']
    group1 = plot_data[plot_data['Group'] == order[0]]['Value']
    group2 = plot_data[plot_data['Group'] == order[1]]['Value']

    p_value = mannwhitneyu(group1, group2, alternative='two-sided').pvalue if len(group1)>0 and len(group2)>0 else 1.0

    title_bbox = dict(facecolor='#fff8c9', alpha=0.8, edgecolor='gray', boxstyle='round,pad=0.2') if p_value < 0.05 else None
    ax.set_title(title, fontsize=14, weight='bold', bbox=title_bbox)

    sns.boxplot(x='Group', y='Value', data=plot_data, ax=ax, palette=['#e74c3c', '#3498db'],
                showfliers=False, order=order, hue='Group', legend=False)
    sns.stripplot(x='Group', y='Value', data=plot_data, ax=ax,
                  color='black', alpha=0.4, jitter=0.2, order=order, s=4)

    ax.text(0.95, 0.95, f'p = {p_value:.3f}', ha='right', va='top', transform=ax.transAxes,
            fontsize=12, bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', pad=0.1))

    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(True, linestyle='--', alpha=0.6)

# ===============================================================
# 3. メイン実行ブロック
# ===============================================================
if __name__ == "__main__":
    file_path = "/content/drive/MyDrive/発表/2025日本眼科AI学会/patientlist'.xlsx"

    if not os.path.exists(file_path):
        print(f"エラー: ファイルが見つかりません: {file_path}")
    else:
        df_processed = preprocess_and_create_variables(file_path)

        variables_to_plot = {
            'ΔMRD-1_Bilateral': 'ΔMRD-1 (両眼)',
            'ΔHertel_Bilateral': 'ΔHertel (両眼)',
            'ΔMRD-2_Bilateral': 'ΔMRD-2 (両眼)',
            'MRD-1_pre_Bilateral': 'MRD-1 pre (両眼)',
            'MRD1_post_Bilateral': 'MRD-1 post (両眼)',
            'MRD-2_pre_Bilateral': 'MRD-2 pre (両眼)',
            'MRD2_post_Bilateral': 'MRD-2 post (両眼)',
            'Hertel_pre_Bilateral': 'Hertel pre (両眼)',
            'Hertel_post_Bilateral': 'Hertel post (両眼)',
            'ΔGorman score': 'ΔGorman score',
            'Gorman score_pre': 'Gorman score pre',
            'Gorman score_post': 'Gorman score post',
            'ΔCAS': 'ΔCAS',
            'CAS_pre': 'CAS pre',
            'CAS_post': 'CAS post',
        }

        n_plots = len(variables_to_plot)
        n_cols = 3
        n_rows = (n_plots + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4.5))
        axes = axes.flatten()

        for i, (col_name, title) in enumerate(variables_to_plot.items()):
            if col_name in df_processed.columns:
                create_comparison_boxplot(axes[i], df_processed, col_name, title)
            else:
                axes[i].set_visible(False)

        for j in range(i + 1, len(axes)):
            axes[j].set_visible(False)

        fig.suptitle('AIスコア群間における臨床指標の比較', fontsize=22, weight='bold')
        plt.tight_layout(rect=[0, 0, 1, 0.96])

        output_filename = 'clinical_comparison_graphs_unified.png'
        plt.savefig(output_filename, dpi=300, bbox_inches='tight')

        print(f"\nグラフが '{output_filename}' として保存されました。")
        plt.show()